In [ ]:
import warnings
warnings.filterwarnings("ignore")
import netCDF4 as nc
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr    
import glob
import os
import cartopy.crs as ccrs
import matplotlib.colors as colors
import pandas as pd
import datetime
from matplotlib import colors
import pymannkendall as mk
from matplotlib.ticker import MaxNLocator
from cartopy import config
import cartopy
import cartopy.crs as ccrs
import climtas
import cmocean
from collections import namedtuple
import dask.array
from datetime import datetime
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import scipy.stats
from scipy import stats
from scipy.stats import ttest_ind 
import seaborn as sns
import sparse
from string import ascii_lowercase
import xarray as xr

In [ ]:
#define colour map
cmap2 = plt.cm.get_cmap("BuGn", 15)

In [ ]:
#heatwave metrix 
HWX_list =["HWA_EHF","HWM_EHF","HWN_EHF","HWF_EHF","HWD_EHF","HWT_EHF"]

#colour map limits 
vmaxlist=[40,20,3.5,17.5,9,80]
vminlist=[0,0,1.5,5,3,40]

#land sea mask 
land_sea = xr.open_dataset("/g/data/w97/sl7808/landmask_accessESM1-5_AUS.nc").sftlf


list2=[1,2,3,4,5,6]


In [ ]:
#open DMI files

d_filenames = sorted(glob.glob("/g/data/w97/sl7808/ALLOUTPUT/DMI_ACCESS/*.nc")) 

file_name = []
f=[]
for d in d_filenames:   
    
    a = xr.open_dataset(d)
    DMI_p=a.sel(time=a.time.dt.month.isin([8,9,10])) #select DMI peak months
    x=DMI_p.tos.groupby('time.year').mean(dim='time')#take the mean of each year
    z=x.rename({'year':'time'})
    del z['time']
    f.insert(-1,z)

pIOD_1=xr.concat(f, 'ensemble') #concat 20 ACCESS ensembles
pIOD=pIOD_1

In [ ]:
#open nino3.4 files (similar to DMI)

nino_Access_filenames = []
nino_Access_filenames = sorted(glob.glob("/g/data/w97/sl7808/ALLOUTPUT/Nino34_ACCESS/all20_nor/*.nc")) 
file_name = []
f=[]


for n in nino_Access_filenames:
        
    a = xr.open_dataset(n)
    a1=a.tos.groupby('time.year').mean(dim='time')#select ENSO peak months #.sel(time=a.time.dt.month.isin([6,7,8,9,10,11])) 
    a2=a1.rename({'year':'time'})
    del a2['time']
    f.insert(-1,a2)

nino=xr.concat(f, 'ensemble')
nino

In [ ]:
#plot one composite for each HWX




#limit for DMI and nino 
p=1
n=-1


for (h,xxx,nnn) in zip (HWX_list,vmaxlist,vminlist):
    
    
    
    #matplotlib stuff
    fig, axes = plt.subplots(3,3, figsize=(15,15), subplot_kw={'projection': ccrs.PlateCarree(central_longitude=0)})
    for i in range(3):
        for j in range(3):
            axes[j,i].axis('off') 
            axes[j,i].coastlines()
            

    
    #open HWX file 
    d_filenames = []
    d_filenames = sorted(glob.glob("/g/data/w97/sl7808/ALLOUTPUT/EHF_year_ACCESS/ehf_year_exp*.nc"))   
    f=[]
    for d in d_filenames:   
        a = xr.open_dataset(d, decode_times=False)
        a1 = a[h] #select which HWX
        a2=a1.where(a1>0) #make sure it's not negative value or NaN
        f.insert(-1,a2)

    HWA=xr.concat(f, 'ensemble') #concat 20 ACCESS ensembles


    
    
    
    

    
    
   # one subplot each
    
    
        
    #Nino+, DMI+
    g=[]
    ggg=[]
    for i in range(20): #for each ensemble
        nino_sel=nino.isel(ensemble=i) #select the ensemble
        pIOD_sel=pIOD.isel(ensemble=i) #select the ensemble
        H1=HWA.isel(ensemble=i).where(land_sea>60) #select the ensemble and mask out the ocean 
        H2=H1.where(pIOD_sel>p).where(nino_sel>p).mean(dim=['time']) # select the years with certain DMI and nino3.4 ranges, take the mean of HWX of that year
        g.insert(-1,H2)
    ggg=xr.concat(g, 'ensemble')  #concat 20 ensembles filtered HWX
    ggg.mean(dim='ensemble').plot(ax=axes[0,0],vmax=xxx,vmin=nnn,cmap=cmap2, add_colorbar=False, transform=ccrs.PlateCarree()) #plot the mean of all years (from all ensembles) match with the DMI and nino range
    
    
    c=nino.where(pIOD_sel>p).where(nino_sel>p).count() #count the number of years from all 20 ensembles that satisfy the DMI and nino3.4 range 
    axes[0,0].set_title( 'Nino+, DMI+' + '  ('+str(c.values)+')', fontsize=10)

    
    #Nino neu, DMI+
    
    g=[]
    ggg=[]
    for i in range(20): 
        nino_sel=nino.isel(ensemble=i)
        pIOD_sel=pIOD.isel(ensemble=i)
        H1=HWA.isel(ensemble=i).where(land_sea>60)
        H2=H1.where(pIOD_sel>p).where(nino_sel>n).where(nino_sel<p).mean(dim=['time'])
        g.insert(-1,H2)
    ggg=xr.concat(g, 'ensemble')  
    #print(ggg)
    ggg.mean(dim='ensemble').plot(ax=axes[0,1],vmax=xxx,vmin=nnn,cmap=cmap2, add_colorbar=False, transform=ccrs.PlateCarree())
    c=nino.where(pIOD_sel>p).where(nino_sel>n).where(nino_sel<p).count()
    axes[0,1].set_title( 'Nino ||, DMI +' + '  ('+str(c.values)+')', fontsize=10) 
    
    
    #Nino-, DMI+
    g=[]
    ggg=[]
    for i in range(20): 
        nino_sel=nino.isel(ensemble=i)
        pIOD_sel=pIOD.isel(ensemble=i)
        H1=HWA.isel(ensemble=i).where(land_sea>60)
        H2=H1.where(pIOD_sel>p).where(nino_sel<n).mean(dim=['time'])
        g.insert(-1,H2)
    ggg=xr.concat(g, 'ensemble')  
    #print(ggg)
    ggg.mean(dim='ensemble').plot(ax=axes[0,2],vmax=xxx,vmin=nnn,cmap=cmap2, add_colorbar=False, transform=ccrs.PlateCarree())
    c=nino.where(pIOD_sel>p).where(nino_sel<n).count()
    axes[0,2].set_title( 'Nino+, DMI-' + '  ('+str(c.values)+')', fontsize=10)      
    
    
    #DMI neutral, Nino +
    
    g=[]
    ggg=[]
    for i in range(20): 
        nino_sel=nino.isel(ensemble=i)
        pIOD_sel=pIOD.isel(ensemble=i)
        H1=HWA.isel(ensemble=i).where(land_sea>60)
        H2=H1.where(pIOD_sel>n).where(pIOD_sel<p).where(nino_sel>p).mean(dim=['time'])
        g.insert(-1,H2)
    ggg=xr.concat(g, 'ensemble')  
    #print(ggg)
    ggg.mean(dim='ensemble').plot(ax=axes[1,0],vmax=xxx,vmin=nnn,cmap=cmap2, add_colorbar=False, transform=ccrs.PlateCarree())
    c=nino.where(pIOD_sel>n).where(pIOD_sel<p).where(nino_sel>p).count()
    axes[1,0].set_title( 'Nino +, DMI ||' + '  ('+str(c.values)+')', fontsize=10)     
    
    #Both neutral
    
    g=[]
    ggg=[]
    for i in range(20): 
        nino_sel=nino.isel(ensemble=i)
        pIOD_sel=pIOD.isel(ensemble=i)
        H1=HWA.isel(ensemble=i).where(land_sea>60)
        H2=H1.where(pIOD_sel>n).where(pIOD_sel<p).where(nino_sel>n).where(nino_sel<p).mean(dim=['time'])
        g.insert(-1,H2)
    ggg=xr.concat(g, 'ensemble') 
    #print(ggg)
    ggg.mean(dim='ensemble').plot(ax=axes[1,1],vmax=xxx,vmin=nnn,cmap=cmap2, add_colorbar=False, transform=ccrs.PlateCarree())
    c=nino.where(pIOD_sel>n).where(pIOD_sel<p).where(nino_sel>n).where(nino_sel<p).count()
    axes[1,1].set_title( 'Nino ||, DMI ||' + '  ('+str(c.values)+')', fontsize=10)           
    
    
    
    #DMI Neu, Nino -
    
    g=[]
    ggg=[]
    for i in range(20): 
        nino_sel=nino.isel(ensemble=i)
        pIOD_sel=pIOD.isel(ensemble=i)
        H1=HWA.isel(ensemble=i).where(land_sea>60)
        H2=H1.where(pIOD_sel>n).where(pIOD_sel<p).where(nino_sel<n).mean(dim=['time'])
        g.insert(-1,H2)
    ggg=xr.concat(g, 'ensemble')  
    #print(ggg)
    ggg.mean(dim='ensemble').plot(ax=axes[1,2],vmax=xxx,vmin=nnn,cmap=cmap2, add_colorbar=False, transform=ccrs.PlateCarree())
    c=nino.where(pIOD_sel>n).where(pIOD_sel<p).where(nino_sel<n).count()
    axes[1,2].set_title( 'Nino -, DMI ||' + '  ('+str(c.values)+')', fontsize=10)           
  
                                     
 
    
    #Nino+, DMI- ************************************* zero?
    g=[]
    ggg=[]
    for i in range(20): 
        nino_sel=nino.isel(ensemble=i)
        pIOD_sel=pIOD.isel(ensemble=i)
        H1=HWA.isel(ensemble=i).where(land_sea>60)
        H2=H1.where(pIOD_sel<n).where(nino_sel>p).mean(dim=['time'])
        g.insert(-1,H2)
    ggg=xr.concat(g, 'ensemble')  
    #print(ggg)
    ggg.mean(dim='ensemble').plot(ax=axes[2,0],vmax=xxx,vmin=nnn,cmap=cmap2, add_colorbar=False, transform=ccrs.PlateCarree())
    c=nino.where(pIOD_sel<n).where(nino_sel>p).count()
    axes[2,0].set_title( 'Nino+, DMI-' + '  ('+str(c.values)+')', fontsize=10)

    
    #DMI -, Nino Neu
    
    g=[]
    ggg=[]
    for i in range(20): 
        nino_sel=nino.isel(ensemble=i)
        pIOD_sel=pIOD.isel(ensemble=i)
        H1=HWA.isel(ensemble=i).where(land_sea>60)
        H2=H1.where(pIOD_sel<n).where(nino_sel>n).where(nino_sel<p).mean(dim=['time'])
        g.insert(-1,H2)
    ggg=xr.concat(g, 'ensemble')  
    #print(ggg)
    ggg.mean(dim='ensemble').plot(ax=axes[2,1],vmax=xxx,vmin=nnn,cmap=cmap2, add_colorbar=False, transform=ccrs.PlateCarree())
    c=nino.where(pIOD_sel<n).where(nino_sel>n).where(nino_sel<p).count()
    axes[2,1].set_title( 'Nino ||, DMI -' + '  ('+str(c.values)+')', fontsize=10)           
    
  
 
    

    #Nino-, DMI-
    g=[]
    ggg=[]
    for i in range(20): 
        nino_sel=nino.isel(ensemble=i)
        pIOD_sel=pIOD.isel(ensemble=i)
        H1=HWA.isel(ensemble=i).where(land_sea>60)
        H2=H1.where(pIOD_sel<n).where(nino_sel<n).mean(dim=['time'])
        g.insert(-1,H2)
    ggg=xr.concat(g, 'ensemble')  
    #print(ggg)
    ggg.mean(dim='ensemble').plot(ax=axes[2,2],vmax=xxx,vmin=nnn,cmap=cmap2, add_colorbar=False, transform=ccrs.PlateCarree())
    c=nino.where(pIOD_sel<n).where(nino_sel<n).count()
    axes[2,2].set_title( 'Nino-, DMI-' + '  ('+str(c.values)+')', fontsize=10)                
                                  
 
                                      
                                
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.suptitle(h + '  all years', fontsize=16)
